In [ ]:
import shutil
import os, pickle
from config import config
from func import load_corpus
from rule_selection_func import rule_selection_func

In [ ]:
validate = load_corpus.load_corpus('PBICSV_validate')
rule_corpus = 'PBICSV'
rule_type_list = ['embed', 'sbert', 'cta', 'doduo', 'pattern', 'pyfunc', 'validator']

# these are for accelerating the computation of sbert and doduo rules
# set them to where the intermediate results are stored (or where they will be stored if they are not available yet)
sbert_dist_val_embeddings_fname = os.path.join(config.dir.storage_root_dir, config.dir.storage_root.sbert, f'{load_corpus.CORPUS_NAME}_dist_val_embeddings.pkl')
doduo_intermediate_result_dir = os.path.join(config.dir.storage_root_dir, config.dir.storage_root.doduo)
doduo_dist_val_scores_fname = f'{load_corpus.CORPUS_NAME}_dist_val_scores.pickle'

rule_outlier_results = rule_selection_func.rule_utility_compute(validate, rule_corpus, rule_type_list, 
                                                                sbert_dist_val_embeddings_fname, doduo_intermediate_result_dir, doduo_dist_val_scores_fname)

In [ ]:
results_file_path = os.path.join(config.dir.storage_root_dir, config.dir.storage_root.validate_results, f'{load_corpus.CORPUS_NAME}_rule_{rule_corpus}_results.pickle')
with open(results_file_path, 'wb') as file:
    pickle.dump(rule_outlier_results, file)

In [ ]:
results_file_path = os.path.join(config.dir.storage_root_dir, config.dir.storage_root.validate_results, f'{load_corpus.CORPUS_NAME}_rule_{rule_corpus}_results.pickle')
with open(results_file_path, 'rb') as file:
    rule_outlier_results = pickle.load(file) 

In [ ]:
rule_fname_list = [os.path.join(config.dir.project_base_dir, config.dir.project_base.sdc_output, f'{rule_corpus}_{rule_type}.pickle') for rule_type in rule_type_list]
rule_list = []
for rule_fname in rule_fname_list:
    with open(rule_fname, 'rb') as file:
        rule_list += pickle.load(file)

In [ ]:
# # coarse_selection

# coarse_select_params = {
#     'cohenh_thres': 0.8,
#     'conf_thres': 0.1,
#     'num_rule_thres': 500,
#     'significance_thres': 0.08/100000,
#     'num_repeat_for_eval': 1000
# }

# randomize_rounding_selection = rule_selection_func.coarse_selection(rule_list, rule_outlier_results, coarse_select_params)
# selected_rules = randomize_rounding_selection()[0]

In [ ]:
# cohenh_thres = coarse_select_params['cohenh_thres']
# cost_thres = coarse_select_params['conf_thres']
# num_rule_thres = coarse_select_params['num_rule_thres']
# directory = f'/datadrive/qixu/coarse_select_rule/rule_{rule_corpus}_cohen_h_{cohenh_thres}_wilson_{cost_thres}_num_rule_{num_rule_thres}'
# if os.path.exists(directory):
#     shutil.rmtree(directory) 
# os.makedirs(directory)
# for i in range(20):
#     selected_rules = randomize_rounding_selection()[0]
#     fine_select_rule_path = os.path.join(directory, f'{i}.pickle')
#     with open(fine_select_rule_path, 'wb') as file:
#         pickle.dump(selected_rules, file)
#     print(fine_select_rule_path)

In [ ]:
# results_file_path = f'/datadrive/qixu/validate_results/{load_corpus.CORPUS_NAME}_rule_{rule_corpus}_results.pickle'
# with open(results_file_path, 'rb') as file:
#     rule_outlier_results = pickle.load(file) 

In [ ]:
# fine_selection

fine_select_params = {
    'cohenh_thres': 0.8,
    'conf_thres': 0.1,
    'num_rule_thres': 500,
    'significance_thres': 0.08/100000,
    'delta': 0.0001,
    'num_repeat_for_eval': 1000,
}

randomize_rounding_selection = rule_selection_func.fine_selection(rule_list, rule_outlier_results, fine_select_params)
selected_rules = randomize_rounding_selection()[0]

In [ ]:
cohenh_thres = fine_select_params['cohenh_thres']
cost_thres = fine_select_params['conf_thres']
num_rule_thres = fine_select_params['num_rule_thres']
directory = os.path.join(config.dir.storage_root_dir, config.dir.storage_root.fine_select_rule,  f'rule_{rule_corpus}_cohen_h_{cohenh_thres}_wilson_{cost_thres}_num_rule_{num_rule_thres}')
if os.path.exists(directory):
    shutil.rmtree(directory) 
os.makedirs(directory)
for i in range(20):
    selected_rules = randomize_rounding_selection()[0]
    fine_select_rule_path = os.path.join(directory, f'{i}.pickle')
    with open(fine_select_rule_path, 'wb') as file:
        pickle.dump(selected_rules, file)
    print(fine_select_rule_path)